In [4]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error


#load cleaned data
df_long = pd.read_csv('../data/cleaned/all_fuel_prices.csv',
parse_dates=['Date'])

# Pivot to wide format
pivot_df = df_long.pivot(index='Date', columns='Fuel Type', values='Price')

model_df = pivot_df.copy()

#add lagged crude oil features
for lag in range(1, 4):
    model_df[f'Crude Oil Lag {lag}'] = model_df['Crude Oil'].shift(lag)


#drop rows with NaNs
model_df_lagged = model_df.dropna()

#identify refined fuels for modeling
refined_fuels = [col for col in model_df.columns if col not in ['Crude Oil', 'Crude Oil Lag 1', 'Crude Oil Lag 2', 'Crude Oil Lag 3']]

#set crude oil lagged features
features = ['Crude Oil', 'Crude Oil Lag 1', 'Crude Oil Lag 2', 'Crude Oil Lag 3']

results = []

for fuel in refined_fuels:
    y = model_df_lagged[fuel]
    X = model_df_lagged[features]

    X_train, X_test,y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    results.append({
        'Fuel Type': fuel,
        'R2 Score': round(r2, 4),
        'MSE': round(mse, 6),
        'Coefficients': dict(zip(features, model.coef_))
    })

#convert to DataFrame for export or display
results_df = pd.DataFrame(results)

#expand coefficient dictionary into columns
coeff_df = results_df.join(results_df['Coefficients'].apply(pd.Series)).drop(columns='Coefficients')

print(coeff_df)

coeff_long = coeff_df.melt(
    id_vars=['Fuel Type', 'R2 Score', 'MSE'],
    value_vars=['Crude Oil', 'Crude Oil Lag 1', 'Crude Oil Lag 2', 'Crude Oil Lag 3'],
    var_name='Feature',
    value_name='Coefficient'
)

# Export for Tableau
coeff_long.to_csv('../data/cleaned/refined_fuel_coefficients.csv', index=False)


               Fuel Type  R2 Score       MSE  Crude Oil  Crude Oil Lag 1  \
0  Conventional Gasoline    0.8411  0.054806   0.025751         0.001467   
1               Jet Fuel    0.8907  0.054650   0.027289         0.001498   
2      No. 2 Heating Oil    0.8250  0.089878   0.027119         0.000388   
3                Propane    0.6208  0.046034   0.015084        -0.003297   
4  RBOB Regular Gasoline    0.7877  0.079895   0.027891         0.001099   
5             ULS Diesel    0.8186  0.097490   0.027684         0.000382   

   Crude Oil Lag 2  Crude Oil Lag 3  
0         0.000589        -0.001352  
1        -0.003094         0.006514  
2        -0.001968         0.005620  
3         0.000715        -0.000443  
4        -0.002171        -0.000835  
5        -0.002352         0.006094  
